In [60]:
try:
    import pandas as pd
    import numpy as np
    import csv
    import json
    import os
    import re
    import matplotlib.pyplot as plt
    import copy
    from tqdm import tqdm
    import pandas as pd
    
    # imports for sparql queries
    from rdflib import Graph
    from SPARQLWrapper import SPARQLWrapper, JSON, N3
    from pprint import pprint
    import ssl


    
except Exception as E:
    print('Some modules are missing')

In [61]:
with open('SMART2022-AT-dbpedia-train.json', 'r') as f:
    at_db_train = json.load(f)
print(len(at_db_train))

36670


In [62]:
with open('SMART2022-AT-dbpedia-test.json', 'r') as f:
    at_db_test = json.load(f)
print(len(at_db_test))

9104


In [63]:
# to check if any null values are present
np.unique(np.array([len(at_db_train[x]['type']) for x in range(len(at_db_train))]))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  19,  23,  30,  31,  47,  81,  93, 627])

### Making all lower case

In [64]:
# Process train data
for e_train in at_db_train:
    e_train['question'] = re.sub(r'[^\w\s]', '', e_train['question'].lower()).strip()

    # stroing unique items
    types = np.array(e_train['type'])
    _, idx = np.unique(types, return_index=True)
    e_train['type'] = types[np.sort(idx)].tolist()


In [65]:
# Process test data
for e_test in at_db_test:
    e_test['question'] = re.sub(r'[^\w\s]', '', e_test['question'].lower()).strip()


### Make lookup table

In [7]:
categories = []
num_types = []
types = []
questions = []

for entry in at_db_train:
    if entry['category']=='resource':
        types.append(entry['type'])

types = [a for sublist in types for a in sublist]
unique_types = np.unique(types)

In [8]:
# some sample queries
#query = 'SELECT DISTINCT ?parentClass WHERE {dbo:Parliament rdfs:subClassOf* ?parentClass. FILTER(CONTAINS("dbpedia")) }'
query = 'SELECT DISTINCT ?parentClass WHERE {dbo:Parliament rdfs:subClassOf* ?parentClass. FILTER(CONTAINS(STR(?parentClass), "dbpedia")) }'
#query = 'SELECT DISTINCT ?parentClass WHERE {dbo:Parliament rdfs:subClassOf* ?parentClass}'
#query = 'SELECT DISTINCT ?parentClass WHERE {dbo:Agent rdfs:subClassOf* ?parentClass. FILTER(?parentClass != dbo:Agent && CONTAINS(STR(?parentClass), "dbpedia")) }'

In [9]:
len(unique_types)

376

In [10]:
owl = '<http://www.w3.org/2002/07/owl#>'
rdfs = '<http://www.w3.org/2000/01/rdf-schema#>'
dbo = '<http://dbpedia.org/ontology/>'

# to disable ssl checks
ssl._create_default_https_context = ssl._create_unverified_context


sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    
lut = []

for i in tqdm(range(len(unique_types))):
    t = unique_types[i]
    query = 'SELECT DISTINCT ?parentClass WHERE {'+t+' rdfs:subClassOf* ?parentClass . FILTER(CONTAINS(STR(?parentClass), "dbpedia"))}'

    for retries in range(3):
        try:
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            qres = sparql.query().convert()
            depth = len(qres['results']['bindings'])

            if depth == 1:
                parent = 'owl:Thing'
            else:
                parent = qres['results']['bindings'][1]
                parent = 'dbo:'+parent['parentClass']['value'].split('/')[-1]

            hierarchy = []
            for x in qres['results']['bindings']:
                hierarchy.append('dbo:'+x['parentClass']['value'].split('/')[-1])
            hierarchy.append('owl:Thing')

            lut.append([t, depth, parent, hierarchy])

            if len(qres) != 0:
                break

        except Exception as E:
            pass
        if retries == 2:
            print(t)

  2%|▉                                          | 8/376 [00:12<10:27,  1.71s/it]

HTTP Error 502: Could not relay message upstream
HTTP Error 502: Could not relay message upstream


 20%|████████▌                                 | 77/376 [03:14<07:21,  1.48s/it]

HTTP Error 502: Could not relay message upstream


 21%|████████▋                                 | 78/376 [03:47<53:20, 10.74s/it]

...

 94%|██████████████████████████████████████▋  | 355/376 [20:29<04:12, 12.00s/it]

HTTP Error 502: Could not relay message upstream


 95%|██████████████████████████████████████▉  | 357/376 [21:02<04:05, 12.95s/it]

HTTP Error 502: Could not relay message upstream


100%|█████████████████████████████████████████| 376/376 [22:02<00:00,  3.52s/it]


In [11]:
query = 'SELECT DISTINCT ?parentClass WHERE {dbo:AnatomicalStructure rdfs:subClassOf* ?parentClass . FILTER(CONTAINS(STR(?parentClass), "dbpedia"))}'
query = 'SELECT DISTINCT ?parentClass WHERE {dbo:Activity rdfs:subClassOf* ?parentClass }'
#query = 'SELECT DISTINCT ?parentClass WHERE {dbo:AnatomicalStructure rdfs:subClassOf* ?parentClass . FILTER(CONTAINS(STR(?parentClass), "dbpedia"))}'


sparql.setQuery(query)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()
qres

{'head': {'link': [], 'vars': ['parentClass']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'parentClass': {'type': 'uri',
     'value': 'http://dbpedia.org/ontology/Activity'}},
   {'parentClass': {'type': 'uri',
     'value': 'http://www.w3.org/2002/07/owl#Thing'}}]}}

In [12]:
t = 'dbo:Archive'
depth = 3
parent = 'dbo:CollectionOfValuables'
hierarchy = ['dbo:Archive', 'dbo:CollectionOfValuables', 'dbo:Work', 'owl:Thing']
lut.append([t, depth, parent, hierarchy])

In [51]:
df = pd.DataFrame(lut, columns=['Type', 'Depth', 'Parent', 'Hierarchy'])

In [66]:
df

,Type,Depth,Parent,Hierarchy
0,dbo:AcademicJournal,4,dbo:PeriodicalLiterature,"[dbo:AcademicJournal, dbo:PeriodicalLiterature..."
1,dbo:AcademicSubject,2,dbo:TopicalConcept,"[dbo:AcademicSubject, dbo:TopicalConcept, owl:..."
2,dbo:Activity,1,owl:Thing,"[dbo:Activity, owl:Thing]"
3,dbo:Actor,6,dbo:Artist,"[dbo:Actor, dbo:Artist, dbo:Person, dbo:Animal..."
4,dbo:AdministrativeRegion,4,dbo:Region,"[dbo:AdministrativeRegion, dbo:Region, dbo:Pop..."
...,...,...,...,...
372,dbo:Wrestler,6,dbo:Athlete,"[dbo:Wrestler, dbo:Athlete, dbo:Person, dbo:An..."
373,dbo:Writer,5,dbo:Person,"[dbo:Writer, dbo:Person, dbo:Animal, dbo:Eukar..."
374,dbo:WrittenWork,2,dbo:Work,"[dbo:WrittenWork, dbo:Work, owl:Thing]"
375,dbo:Zoo,2,dbo:ArchitecturalStructure,"[dbo:Zoo, dbo:ArchitecturalStructure, owl:Thing]"


In [67]:
df = df[~((df['Type']=='dbo:Archive') & (df['Depth']==1))]

In [68]:
len(df)

376

### Check if first type is always the deepest level

In [69]:
for entry in at_db_train:
    if entry['category']=='resource':
        types = entry['type']
        #if np.sum(np.array(types)=='dbo:Archive'):
        #    print(entry)
        #if np.sum(np.array(types)=='dbo:Location'):
        #    print(entry)
        if np.sum(np.array(types)=='dbo:RaceHorse'):
            print(entry)

# Valid: 'dbo:Archive', 'dbo:CollectionOfValuables'

{'id': 25415, 'question': 'who did viggo mortensen play in lord of the rings', 'category': 'resource', 'type': ['dbo:Agent', 'dbo:FictionalCharacter', 'dbo:RaceHorse']}
{'id': 44915, 'question': 'who was born in columbus', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Building', 'dbo:Agent', 'dbo:HorseTrainer', 'dbo:RaceHorse']}


In [70]:
for entry in at_db_train:
    if entry['category']=='resource':
        if 'dbo:Location' in entry['type']:
            entry['type'].remove('dbo:Location')
        if 'dbo:RaceHorse' in entry['type']:
            entry['type'].remove('dbo:RaceHorse')
        depths = [df.loc[df['Type'] == x, 'Depth'].iloc[0] for x in entry['type']]
        if np.sum(depths[1:]>depths[0]):
            print(depths, entry)
            
# Comment: First type is not at the deepest level.
# Also types do not belong to same category

[5, 6, 5, 4, 1] {'id': 35, 'question': 'which key person of clinton foundation is also the president of jim sasser', 'category': 'resource', 'type': ['dbo:OfficeHolder', 'dbo:President', 'dbo:Politician', 'dbo:Person', 'dbo:Agent']}
[3, 4, 3, 2, 1] {'id': 371, 'question': 'which constructors team won the 1967 mexican grand prix', 'category': 'resource', 'type': ['dbo:Company', 'dbo:FormulaOneTeam', 'dbo:SportsTeam', 'dbo:Organisation', 'dbo:Agent']}
[3, 4, 2, 1] {'id': 995, 'question': 'what is the name of the association football club whose head coach is clarence seedorf', 'category': 'resource', 'type': ['dbo:SportsClub', 'dbo:SoccerClub', 'dbo:Organisation', 'dbo:Agent']}
[3, 4, 2, 1] {'id': 1063, 'question': 'what is the type of naval station bremerton', 'category': 'resource', 'type': ['dbo:MilitaryUnit', 'dbo:PublicTransitSystem', 'dbo:Organisation', 'dbo:Agent']}
[2, 3, 2, 1] {'id': 1125, 'question': 'for what work did poul anderson receive the prometheus award  hall of fame', '

[1, 2, 1] {'id': 12504, 'question': 'which is the triumphal arch that jean chalgrin was the architect and contains the word arc in its name', 'category': 'resource', 'type': ['dbo:ArchitecturalStructure', 'dbo:Monument', 'dbo:Place']}
[3, 4, 3, 2, 1] {'id': 12667, 'question': 'what is the origin of the musical artist of a todo romantic style', 'category': 'resource', 'type': ['dbo:Country', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace', 'dbo:Place']}
[2, 4, 1] {'id': 12751, 'question': 'to which series does lost special belong to', 'category': 'resource', 'type': ['dbo:FictionalCharacter', 'dbo:Person', 'dbo:Agent']}
[2, 4, 1] {'id': 13241, 'question': 'which political party got elected in first legislative assembly of uttar pradesh', 'category': 'resource', 'type': ['dbo:FictionalCharacter', 'dbo:Person', 'dbo:Agent']}
[1, 2, 1] {'id': 13313, 'question': 'where did david livingstone die', 'category': 'resource', 'type': ['dbo:ArchitecturalStructure', 'dbo:Monument', 'dbo:Place']}

[2, 4, 1, 3] {'id': 22154, 'question': 'who will david beckham play for in 2013', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:SoccerClub', 'dbo:Agent', 'dbo:SportsClub']}
[1, 4, 3, 3, 2] {'id': 22157, 'question': 'what county is novato california in', 'category': 'resource', 'type': ['dbo:Place', 'dbo:AdministrativeRegion', 'dbo:Settlement', 'dbo:Region', 'dbo:PopulatedPlace']}
[1, 3, 2] {'id': 22159, 'question': 'where did hernando cortez die', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[1, 4, 3, 2] {'id': 22164, 'question': 'which city held the summer olympics twice', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[4, 5, 1] {'id': 22165, 'question': 'what inspired langston hughes to write poetry', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Writer', 'dbo:Agent']}
[1, 3, 4, 2] {'id': 22168, 'question': 'where did henry knox die', 'category': 'resource', 'type': ['dbo:P

[1, 4, 3, 2] {'id': 22718, 'question': 'where does allen iverson live now', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[1, 4, 3, 2] {'id': 22719, 'question': 'where was dwight howard born', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[1, 6, 3, 2] {'id': 22721, 'question': 'where is alexandria located', 'category': 'resource', 'type': ['dbo:Place', 'dbo:MusicalArtist', 'dbo:Country', 'dbo:PopulatedPlace']}
[2, 4, 1, 3] {'id': 22723, 'question': 'what team does chris humphrey play for', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:SoccerClub', 'dbo:Agent', 'dbo:SportsClub']}
[1, 4, 3, 3, 2, 4] {'id': 22733, 'question': 'where is jack daniels tennessee whiskey made', 'category': 'resource', 'type': ['dbo:Place', 'dbo:AdministrativeRegion', 'dbo:Settlement', 'dbo:Region', 'dbo:PopulatedPlace', 'dbo:Village']}
[4, 5, 1, 5] {'id': 22735, 'question': 'who was governor of

[2, 4, 1, 3] {'id': 23203, 'question': 'what club did aguero play for before man city', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:SoccerClub', 'dbo:Agent', 'dbo:SportsClub']}
[1, 3, 2] {'id': 23206, 'question': 'what did the habsburg family control', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Country', 'dbo:PopulatedPlace']}
[3, 2, 1, 4] {'id': 23208, 'question': 'what college did michael jordan play basketball for', 'category': 'resource', 'type': ['dbo:EducationalInstitution', 'dbo:Organisation', 'dbo:Agent', 'dbo:University']}
[4, 5, 1, 5] {'id': 23210, 'question': 'who was lincoln s vice president when he died', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Politician', 'dbo:Agent', 'dbo:OfficeHolder']}
[1, 4, 3, 2] {'id': 23211, 'question': 'where did pope benedict xvi live', 'category': 'resource', 'type': ['dbo:Place', 'dbo:AdministrativeRegion', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[1, 4, 4, 3, 3, 2] {'id': 23213, 'question': 'where is tommy em

...

[1, 4, 3, 2, 4, 3] {'id': 48759, 'question': 'where did philip b thompson jr die', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace', 'dbo:AdministrativeRegion', 'dbo:Region']}
[2, 3, 1] {'id': 48761, 'question': 'what album is by  café tacuba', 'category': 'resource', 'type': ['dbo:MusicalWork', 'dbo:Album', 'dbo:Work']}
[1, 6, 3, 2] {'id': 48762, 'question': 'what country is welcome to the show from', 'category': 'resource', 'type': ['dbo:Place', 'dbo:MusicalArtist', 'dbo:Country', 'dbo:PopulatedPlace']}
[1, 3, 2] {'id': 48763, 'question': 'what constellation is ursa major found in', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Star', 'dbo:CelestialBody']}
[1, 3, 4, 2] {'id': 48764, 'question': 'where in the united states was martin johnson born in', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Settlement', 'dbo:Town', 'dbo:PopulatedPlace']}
[3, 1, 3, 4, 2] {'id': 48766, 'question': 'where did battle of reading happen', 'cate

[1, 4, 3, 2] {'id': 49150, 'question': 'where in california was suzanne ridgeway born', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[2, 3, 4, 1, 2] {'id': 49151, 'question': 'which country filmed when hell broke loose', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:Country', 'dbo:Person', 'dbo:Place', 'dbo:PopulatedPlace']}
[1, 3, 2, 3, 4, 3, 3] {'id': 49152, 'question': 'what country was the kiss in the tunnel filmed in', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Country', 'dbo:PopulatedPlace', 'dbo:Settlement', 'dbo:AdministrativeRegion', 'dbo:Region', 'dbo:Island']}
[2, 3, 4, 1, 2] {'id': 49155, 'question': 'where is william stewart halsted born', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:Country', 'dbo:Person', 'dbo:Place', 'dbo:PopulatedPlace']}
[1, 4, 3, 2] {'id': 49161, 'question': 'which city was al wickland born', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement

[2, 3, 4, 1, 2] {'id': 49588, 'question': 'which country produces city of water', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:Country', 'dbo:Person', 'dbo:Place', 'dbo:PopulatedPlace']}
[1, 6, 3, 2, 3, 4] {'id': 49589, 'question': 'which country is halfdan rasmussen from', 'category': 'resource', 'type': ['dbo:Place', 'dbo:MusicalArtist', 'dbo:Country', 'dbo:PopulatedPlace', 'dbo:Settlement', 'dbo:Village']}
[4, 5, 1] {'id': 49590, 'question': 'who is someone that was born in berlin', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Artist', 'dbo:Agent']}
[1, 6, 3, 2] {'id': 49591, 'question': 'where is the uyarthiru 420 from', 'category': 'resource', 'type': ['dbo:Place', 'dbo:MusicalArtist', 'dbo:Country', 'dbo:PopulatedPlace']}
[1, 4, 3, 2] {'id': 49594, 'question': 'where was joseph griffo born', 'category': 'resource', 'type': ['dbo:Place', 'dbo:City', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[4, 5, 6, 1] {'id': 49595, 'question': 'what artist sings opera', 'categ

[4, 5, 6, 1] {'id': 50006, 'question': 'name a singer', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Artist', 'dbo:MusicalArtist', 'dbo:Agent']}
[2, 3, 4, 1, 2] {'id': 50007, 'question': 'where was sing a song of six pants originally released', 'category': 'resource', 'type': ['dbo:Organisation', 'dbo:Country', 'dbo:Person', 'dbo:Place', 'dbo:PopulatedPlace']}
[4, 5, 1, 6] {'id': 50008, 'question': 'who plays midfielder', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Athlete', 'dbo:Agent', 'dbo:SoccerPlayer']}
[1, 3, 2] {'id': 50009, 'question': 'where was chocoball mukai born', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Settlement', 'dbo:PopulatedPlace']}
[4, 5, 1] {'id': 50011, 'question': 'what is the name of the independent filmmaker director of portrait of jason', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Artist', 'dbo:Agent']}
[1, 4, 3, 4, 2] {'id': 50012, 'question': 'what vietnamese city was dat phan born in', 'category': 'resource', 'type': ['dbo

[1, 6, 3, 2] {'id': 50406, 'question': 'what is the nation to which yann songoo belongs', 'category': 'resource', 'type': ['dbo:Place', 'dbo:MusicalArtist', 'dbo:Country', 'dbo:PopulatedPlace']}
[1, 3, 4, 2] {'id': 50412, 'question': 'where did karl august prince of waldeck and pyrmont die', 'category': 'resource', 'type': ['dbo:Place', 'dbo:Settlement', 'dbo:Town', 'dbo:PopulatedPlace']}
[4, 6, 1] {'id': 50413, 'question': 'who is an artist  signed by rca records', 'category': 'resource', 'type': ['dbo:Person', 'dbo:MusicalArtist', 'dbo:Agent']}
[4, 1, 6, 6, 5] {'id': 50414, 'question': 'who is a notable figure that was born in buenos aires', 'category': 'resource', 'type': ['dbo:Person', 'dbo:Agent', 'dbo:SoccerManager', 'dbo:SoccerPlayer', 'dbo:SportsManager']}
[1, 1, 2, 3] {'id': 50418, 'question': 'what was the location of elizabeth stuarts death', 'category': 'resource', 'type': ['dbo:Place', 'dbo:ArchitecturalStructure', 'dbo:Building', 'dbo:Castle']}
[1, 3, 2] {'id': 50419, 'qu

### Replace GT Types with hierarchy based on deepest type

In [71]:
for entry in at_db_train:

    if entry['category']=='resource':
        depths = np.array([df.loc[df['Type'] == x, 'Depth'].iloc[0] for x in entry['type']])

        x = np.array(entry['type'])[depths==depths.max()][0]
        # Some types are at same depths -> Take only one type in such cases
            
        hierarchy = df.loc[df['Type'] == x, 'Hierarchy'].iloc[0][:-1]
        entry['type'] = hierarchy

In [72]:
df.loc[df['Type'] == 'dbo:Company', 'Hierarchy'].iloc[0][:-1]
# FILTER in SPARQL query is needed

['dbo:Company', 'dbo:Organisation', 'dbo:Agent']

In [73]:
at_db_train

[{'id': 0,
  'question': 'was jacqueline kennedy onassis a follower of melkite greek catholic church',
  'category': 'boolean',
  'type': ['boolean']},
 {'id': 1,
  'question': 'what is the name of the opera based on twelfth night',
  'category': 'resource',
  'type': ['dbo:Opera', 'dbo:MusicalWork', 'dbo:Work']},
 {'id': 2,
  'question': 'when did lena horne receive the grammy award for best jazz vocal album',
  'category': 'literal',
  'type': ['date']},
 {'id': 3,
  'question': 'do prince harry and prince william have the same parents',
  'category': 'boolean',
  'type': ['boolean']},
 {'id': 5,
  'question': 'which is the hierarchical braininfo id of the frontal lobe',
  'category': 'literal',
  'type': ['string']},
 {'id': 6,
  'question': 'what is the federated state located in the weimar republic',
  'category': 'resource',
  'type': ['dbo:State', 'dbo:PopulatedPlace', 'dbo:Place']},
 {'id': 7,
  'question': 'did buddhism was named after the immigration of united states',
  'cat

In [74]:
with open('cleaned_SMART2022-AT-dbpedia-train.json', 'w') as f:
    json.dump(at_db_train, f)

In [75]:
with open('cleaned_SMART2022-AT-dbpedia-test.json', 'w') as f:
    json.dump(at_db_test, f)

### Cleaned data analysis

In [76]:
with open('cleaned_SMART2022-AT-dbpedia-train.json', 'r') as f:
    at_db_train = json.load(f)

In [77]:
word = []
for x in range(len(at_db_train)):
    q, t = at_db_train[x]['question'], at_db_train[x]['category']
    if t == 'literal':
        #print(q)
        word.append(q.split(' ')[0])
    else:
        word.append('others')

uq_word, qid, ct = np.unique(np.array(word), return_counts=True, return_index=True)
print(uq_word)
print([uq_word[x] for x in range(len(uq_word)) if ct[x]>5])

['according' 'anatoly' 'anhydrous' 'anna' 'at' 'bananas' 'based' 'bonar'
 'by' 'cable' 'can' 'canada' 'charles' 'chester' 'coriandrum' 'count'
 'cowboy' 'current' 'did' 'do' 'does' 'dortmund' 'during' 'eiffel'
 'england' 'enumerate' 'ernest' 'eros' 'estimate' 'ethanol' 'find' 'for'
 'foxconn' 'from' 'george' 'get' 'give' 'graduates' 'gregg' 'harold'
 'hiram' 'how' 'hungary' 'i' 'in' 'is' 'java' 'jleague' 'joe' 'journals'
 'jules' 'karlsruhe' 'kendall' 'larginine' 'liscence' 'list' 'livermorium'
 'louis' 'marge' 'marie' 'mario' 'martin' 'maryland' 'may' 'melbourne'
 'mention' 'mig21' 'mike' 'mona' 'mount' 'muhammad' 'murray' 'name'
 'neymar' 'north' 'on' 'others' 'over' 'paul' 'pennsylvania' 'people'
 'please' 'provide' 'reading' 'richard' 'ryan' 'since' 'sputnik'
 'stockholm' 'swimrankingnet' 'ted' 'tell' 'terry' 'texas' 'the' 'there'
 'this' 'through' 'to' 'tolyatti' 'total' 'until' 'vehicles' 'was'
 'watchmaker' 'what' 'whatis' 'whats' 'when' 'where' 'which' 'who' 'why'
 'wich' 'will

In [83]:
word = []
for x in range(len(at_db_train)):
    q, t = at_db_train[x]['question'], at_db_train[x]['category']
    if t == 'boolean':
        #print(q)
        word.append(q.split(' ')[0])
    else:
        word.append('others')

uq_word, qid, ct = np.unique(np.array(word), return_counts=True, return_index=True)
print([uq_word[x] for x in range(len(uq_word)) if ct[x]>3])

['are', 'did', 'do', 'does', 'has', 'is', 'others', 'was', 'were', 'what']


In [79]:
types = []

for entry in at_db_train:
    if entry['category']=='resource':
        types.append(entry['type'])

types = [a for sublist in types for a in sublist]
unique_types = np.unique(types)

In [80]:
t, c = np.unique(types, return_counts=True)
ind = np.argsort(-c)
[f'{t[ind[i]]}: {c[ind[i]]}' for i in range(len(t))]

['dbo:Species: 10900',
 'dbo:Eukaryote: 10897',
 'dbo:Animal: 10862',
 'dbo:Person: 10767',
 'dbo:Place: 7048',
 'dbo:PopulatedPlace: 6233',
 'dbo:Settlement: 4156',
 'dbo:Work: 3558',
 'dbo:City: 3228',
 'dbo:Agent: 2964',
 'dbo:Organisation: 2840',
 'dbo:Artist: 2096',
 'dbo:MusicalArtist: 1961',
 'dbo:TopicalConcept: 1945',
 'dbo:Genre: 1714',
 'dbo:MusicGenre: 1714',
 'dbo:Athlete: 1404',
 'dbo:Film: 1135',
 'dbo:Country: 1056',
 'dbo:Language: 1005',
 'dbo:Company: 922',
 'dbo:Region: 806',
 'dbo:AdministrativeRegion: 793',
 'dbo:WrittenWork: 712',
 'dbo:MusicalWork: 703',
 'dbo:Group: 638',
 'dbo:Band: 638',
 'dbo:Book: 632',
 'dbo:Software: 622',
 'dbo:Album: 603',
 'dbo:VideoGame: 563',
 'dbo:ArchitecturalStructure: 467',
 'dbo:Politician: 441',
 'dbo:CelestialBody: 412',
 'dbo:EducationalInstitution: 401',
 'dbo:Writer: 360',
 'dbo:SoccerPlayer: 359',
 'dbo:NaturalPlace: 357',
 'dbo:RugbyPlayer: 344',
 'dbo:PersonFunction: 341',
 'dbo:Award: 291',
 'dbo:Disease: 289',
 'dbo:Ev

In [81]:
np.unique(num_types)

array([], dtype=float64)

In [82]:
at_db_train

[{'id': 0,
  'question': 'was jacqueline kennedy onassis a follower of melkite greek catholic church',
  'category': 'boolean',
  'type': ['boolean']},
 {'id': 1,
  'question': 'what is the name of the opera based on twelfth night',
  'category': 'resource',
  'type': ['dbo:Opera', 'dbo:MusicalWork', 'dbo:Work']},
 {'id': 2,
  'question': 'when did lena horne receive the grammy award for best jazz vocal album',
  'category': 'literal',
  'type': ['date']},
 {'id': 3,
  'question': 'do prince harry and prince william have the same parents',
  'category': 'boolean',
  'type': ['boolean']},
 {'id': 5,
  'question': 'which is the hierarchical braininfo id of the frontal lobe',
  'category': 'literal',
  'type': ['string']},
 {'id': 6,
  'question': 'what is the federated state located in the weimar republic',
  'category': 'resource',
  'type': ['dbo:State', 'dbo:PopulatedPlace', 'dbo:Place']},
 {'id': 7,
  'question': 'did buddhism was named after the immigration of united states',
  'cat